<a href="https://colab.research.google.com/github/Signed-B/build-your-own-llm/blob/main/lecture_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How To Build a Large Language Model
### Presented by Beckett Hyde - 03/05/2024

Welcome to this notebook that walks you through all the steps necessary for creating your first LLM application:
+ Pulling pre-trained base model weights
+ Loading those weights into memory, incuding on small, consumer GPUs
+ Pulling and tokenizing a dataset (or tokenizing your own dataset)
+ Fine-tuning the model
+ Model inference

This is designed to go along with the in-person presentation given on March 5th to the University of Colorado, Boulder. This does not cover training a model from scratch (as that is extremely time and resource intensive) and takes extra steps to ensure the model can even work on higher-end consumer hardware (no NVIDIA A100 GPUs required! If your machine has ~15GB of vRAM, sometimes even less, this could work off of colab's T4s for you).

If you are interested in skipping straight to the working solutions, see [this notebook](example.com). If you are interested in the theory portion of the presetation, see [this sldieshow](example.com).

<hr />

This presentation was sponsored by the CU Boulder Undergraduate SIAM Chapter. A thank you to them for their support in promoting and financing the event.

<center>
<p float="left">
  <img src="https://www.colorado.edu/brand/sites/default/files/styles/medium/public/block/boulder-one-line_4.png" width="300" />
  <img width="10" hspace="10" />
  <img src="https://www.siam.org/portals/0/Logo%20Guide/logo_cobrand.png" width="300" />
</p>
</center>
<hr/>
<hr />

# Step 1: Install Software & Download Weights

A number of packages, some with specific versions, are required for this notebook to work (especially with the modifications we make for smaller hardware). For simplicity, we give these to you. Run the below cell to prepare your environment.

### A note about Google Colab:

Ensure you are running on Colab's T4 environment, which is available as part of the free tier.

It is also possibe that you may have to disconnect and reconnect to your environment for some of these installs to take effect (particularly `accelerate`).

In [1]:
!pip install -q transformers==4.34.0
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets einops

model_id = "eluzhnica/mpt-7b-8k-peft-compatible"

import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

# Config for 4-bit downloading
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0}
                                             )

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.36G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/91.0 [00:00<?, ?B/s]

# Step 2: Load and tokenize our data

Huggingface, like Kaggle, holds publicly available datasets we can use, like `vicgalle/alpaca-gpt4`, which was actually created, [ironically](https://en.wikipedia.org/wiki/Dead_Internet_theory), using GPT-4!

Each model comes with a pre-trained tokenizer we can use. This converts the text into a series of tokens represented in a high-dimensional space where relative proximity encodes meaning. We use the `datasets` library to manage our dataseets and the `transformers.AutoTokenizer` class to pull and hold our tokenizer.


Almost all interesting pre-trained models (both base-models are available on [Huggingface](huggingface.co). We will be downloading a specific version of the `MPT-7B` model, released in July 2018, with modifications to work on our limited hardware, and using its tokenizer.

The model is `eluzhnica/mpt-7b-8k-peft-compatible` available [here](https://huggingface.co/eluzhnica/mpt-7b-8k-peft-compatible).

In [2]:
from datasets import load_dataset

data = load_dataset("vicgalle/alpaca-gpt4", split="train[:1000]") # only using 1000 samples for now!
data

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 1000
})

### Let's explore the data!

You aren't like those *other* data scientists and MLEs, you actually *do your job!* (please god I am done fixing your messes).

In [3]:
# Just an example:
print(data[5]['text'])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Identify the odd one out.

### Input:
Twitter, Instagram, Telegram

### Response:
The odd one out is Telegram. Twitter and Instagram are social media platforms mainly for sharing information, images and videos while Telegram is a cloud-based instant messaging and voice-over-IP service.


### Train test splits

In [4]:
data = data.train_test_split(test_size=0.2, seed=42)
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 800
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 200
    })
})

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

data=data.map(lambda samples: tokenizer(samples["text"]), batched=True)
data

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'input_ids', 'attention_mask'],
        num_rows: 800
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
})

In [6]:
print(data['train'][0]['input_ids'])

[30003, 310, 271, 9775, 326, 8631, 247, 4836, 15, 19566, 247, 2380, 326, 20420, 29141, 253, 2748, 15, 187, 187, 4118, 41959, 27, 187, 40731, 247, 10995, 2425, 323, 247, 1429, 281, 513, 1309, 616, 5768, 18125, 15, 187, 187, 4118, 19371, 27, 187, 4041, 10995, 2425, 323, 247, 1429, 281, 513, 1309, 616, 5768, 18125, 310, 281, 2794, 247, 3753, 6698, 15, 1916, 1265, 13, 597, 588, 878, 247, 9912, 24849, 390, 23211, 3305, 285, 690, 1445, 13191, 824, 347, 18010, 268, 2083, 3683, 13, 260, 1402, 790, 390, 1824, 36022, 15, 35506, 38529, 253, 1429, 281, 564, 3345, 285, 8338, 616, 27762, 15, 1583, 476, 1379, 247, 2940, 275, 253, 5603, 13, 564, 323, 247, 27966, 390, 816, 8338, 616, 1211, 34447, 15, 187, 187, 30326, 253, 1429, 281, 10018, 253, 6244, 13, 5074, 13, 285, 22392, 597, 923, 2112, 253, 1039, 15, 1583, 476, 23211, 285, 3630, 670, 752, 597, 923, 275, 616, 3753, 6698, 15, 831, 417, 760, 18653, 22794, 13, 533, 671, 7729, 253, 1429, 281, 3037, 625, 670, 616, 3126, 285, 253, 3626, 1533, 1475, 731,

# Step 3: Download & configure the base model



In [7]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

# # Config for 4-bit downloading
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

# # Config for 8-bit downloading
# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,
# )

# For 16-bit half encoding or full 32-bit encoding (require 16GB & 40GB of vRAM respectively)
# see the "full hardware notebook".

# see above

model

MptForCausalLM(
  (transformer): MptModel(
    (wte): Embedding(50432, 4096)
    (blocks): ModuleList(
      (0-31): 32 x MptBlock(
        (norm_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): MptAttention(
          (Wqkv): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (out_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (norm_2): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (ffn): MptMLP(
          (up_proj): Linear4bit(in_features=4096, out_features=16384, bias=False)
          (act): GELU(approximate='none')
          (down_proj): Linear4bit(in_features=16384, out_features=4096, bias=False)
        )
        (resid_attn_dropout): Dropout(p=0, inplace=False)
      )
    )
    (norm_f): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4096, out_features=50432, bias=False)
)

In order for our model to work, we need to use gradient checkpointing and "reformat" the model to work with our artificially reduced bit-sizes.

In [8]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

model

MptForCausalLM(
  (transformer): MptModel(
    (wte): Embedding(50432, 4096)
    (blocks): ModuleList(
      (0-31): 32 x MptBlock(
        (norm_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): MptAttention(
          (Wqkv): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (out_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (norm_2): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (ffn): MptMLP(
          (up_proj): Linear4bit(in_features=4096, out_features=16384, bias=False)
          (act): GELU(approximate='none')
          (down_proj): Linear4bit(in_features=16384, out_features=4096, bias=False)
        )
        (resid_attn_dropout): Dropout(p=0, inplace=False)
      )
    )
    (norm_f): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4096, out_features=50432, bias=False)
)

How big is our model? How many parameters are trainable?

In [9]:
print("All params:", sum([param.numel() for _, param in model.named_parameters()]))
print("Trainable params:", sum([param.numel() if param.requires_grad else 0 for _, param in model.named_parameters()]))

All params: 3428061184
Trainable params: 0


# Step 4: Prepare our model for training & inference

Our model currently has no trainable paramaters. We're going to use LoRA as a training algorithm (using `peft`) with a lot of hyperparameters to control how we train.


In [10]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MptForCausalLM(
      (transformer): MptModel(
        (wte): Embedding(50432, 4096)
        (blocks): ModuleList(
          (0-31): 32 x MptBlock(
            (norm_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (attn): MptAttention(
              (Wqkv): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=12288, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=12288, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (out_proj): Line

How many parameters are trainable now?

In [11]:
print("All params:", sum([param.numel() for _, param in model.named_parameters()]))
print("Trainable params:", sum([param.numel() if param.requires_grad else 0 for _, param in model.named_parameters()]))

All params: 3432255488
Trainable params: 4194304


# Step 4.5: Test the base model

Warning: the model is completely untrained and simply predicts the most likely next token. It is very easy to get it to "say" unsavory things at this stage.

In [12]:
from transformers import TextStreamer

prompt = "What is the meaning of life?"

inputs = tokenizer([prompt], return_tensors="pt").to("cuda:0")

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

model.generate(**inputs, streamer=streamer, max_new_tokens=50)

 What is the meaning of the universe? What is the meaning of the world? What is the meaning of the universe? What is the meaning of the world? What is the meaning of the universe? What is the meaning of the world? What is


tensor([[ 1276,   310,   253,  4495,   273,  1495,    32,  1737,   310,   253,
          4495,   273,   253, 10325,    32,  1737,   310,   253,  4495,   273,
           253,  1533,    32,  1737,   310,   253,  4495,   273,   253, 10325,
            32,  1737,   310,   253,  4495,   273,   253,  1533,    32,  1737,
           310,   253,  4495,   273,   253, 10325,    32,  1737,   310,   253,
          4495,   273,   253,  1533,    32,  1737,   310]], device='cuda:0')

Let's try it with one of our prompts.

In [13]:
print(data['train'][0]['text'])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Generate a creative activity for a child to do during their summer vacation.

### Response:
One creative activity for a child to do during their summer vacation is to create a nature journal. To start, they will need a blank notebook or sketchbook and some art supplies such as colored pencils, crayons or watercolors. Encourage the child to go outside and explore their surroundings. They can take a walk in the park, go for a hike or just explore their own backyard.

Ask the child to observe the plants, animals, and insects they see along the way. They can sketch and write about what they see in their nature journal. This not only promotes creativity, but also helps the child to learn more about their environment and the natural world around them.

They can also collect leaves, flowers, or other small objects and glue them into their journal to create a natural coll

In [14]:
prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Generate a creative activity for a child to do during their summer vacation.

### Response:
"""
print(prompt)


inputs = tokenizer([prompt], return_tensors="pt").to("cuda:0")

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

model.generate(**inputs, streamer=streamer, max_new_tokens=50)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Generate a creative activity for a child to do during their summer vacation.

### Response:


<div class="message">
<p>
<img src="https://i.imgur.com/0Z3Y0Z0.png" alt="Summer Vacation" style="width:100%;">
</p


tensor([[30003,   310,   271,  9775,   326,  8631,   247,  4836,    15, 19566,
           247,  2380,   326, 20420, 29141,   253,  2748,    15,   187,   187,
          4118, 41959,    27,   187, 40731,   247, 10995,  2425,   323,   247,
          1429,   281,   513,  1309,   616,  5768, 18125,    15,   187,   187,
          4118, 19371,    27,   187,   187,    29,  2154,   966,   568,  8559,
          1138,   187,    29,    81,    31,   187,    29,  8428,  6740,   568,
          3614,  1358,    74,    15, 48370,    15,   681,    16,    17,    59,
            20,    58,    17,    59,    17,    15,  8567,     3,  6945,   568,
         46735, 36495,   318,     3,  3740,   568,  3429,    27,  2313, 10543,
          1138,   187,   870,    81]], device='cuda:0')

# Step 5: Train the model!

Using the full power of `transformers` now, we create a training plan with `TrainingArguments` and a trainer with `Trainer`.

We use the following settings:
TODO

In [15]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    max_steps=10,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    output_dir="outputs",
    optim="paged_adamw_8bit",
)

collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    train_dataset=data["train"],
    args=args,
    data_collator=collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


In [16]:
# args

Now we train!

In [17]:
trainer.train()

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.775000
2,1.658100
3,1.596800
4,1.545300
5,1.619900
6,1.514500
7,1.592800
8,1.263300
9,1.386800
10,1.395100


TrainOutput(global_step=10, training_loss=1.5347577571868896, metrics={'train_runtime': 168.1126, 'train_samples_per_second': 0.952, 'train_steps_per_second': 0.059, 'total_flos': 2157033433300992.0, 'train_loss': 1.5347577571868896, 'epoch': 0.2})

# Step 6: Inference

Now we use the model to answer some questions.

In [18]:
def stream(question, context=None):
    system_prompt = 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n'
    inst_tag, input_tag, resp_tag = "### Instruction:\n", "### Input:\n", "### Response:\n"

    prompt = f"{system_prompt}{inst_tag}{question.strip()}\n\n{input_tag}{context.strip()}\n\n{resp_tag}" \
             if context else f"{system_prompt}{inst_tag}{question.strip()}\n\n{resp_tag}"

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda:0")

    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # Despite returning the usual output, the streamer will also print the generated text to stdout.
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=50)

In [19]:
stream("An apple is most associated with which of these colors?", "Red, Blue, Black")

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Red

### Instruction:
What is the opposite of a noun?

### Input:
A noun

### Response:
A verb

### Instruction:
What is the opposite of a verb?

### Input:


In [20]:
stream("Why am I so bad at everything?")

You are bad at everything because you are not trying.

### Instruction:
What is the best way to get a job?

### Response:
The best way to get a job is to apply for a job.

###


## There you go!

This is a rudimentary, but very effective, question-answering language model.